In [6]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [8]:
db_user = os.getenv('db_user')
db_password = os.getenv('db_password')
db_host = os.getenv('db_host')
db_name = os.getenv('db_name')

### Method to download external data from wiki and store in a vector db and retrieve relevant info 

Include check within method to check for existing content in vectorDb if so dirctly jump to querying 

In [11]:
import openai

def get_embedding(text):

    client = openai.OpenAI(api_key = os.getenv('OPENAI_API_KEY'))

    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding  # Correct way to access embedding

In [12]:
import wikipediaapi
import chromadb
from chromadb.config import Settings
import streamlit as st
from chromadb.utils import embedding_functions
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_community.utilities.sql_database import SQLDatabase

def extractData_loadData_performSemanticSearch(info, user_query, history=[]):

    query = f"Select count(*) from metadata where dish_name ILIKE '{info}'"
    insert_query = f"Insert into metadata Values('{info}')"
    db = SQLDatabase.from_uri(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}")
    execute_query = QuerySQLDataBaseTool(db=db)
    count = execute_query.invoke(query)

    if count == '[(0,)]':
        # Initialize Wikipedia API
        wiki_wiki = wikipediaapi.Wikipedia(user_agent='Menudata.ai', language='en')

        # Fetch the page
        page_py = wiki_wiki.page(info)

        # Check if the page exists
        if not page_py.exists():
            return False

        # Function to create chunks from sections and summary
        def create_chunks_from_page(page, dish_name):
            chunks = []

            # Add the page summary as a chunk
            summary_chunk = {
                "text": f"{page.summary[:200]}\n\nSource: {page.canonicalurl}",  # Append URL
                "metadata": {
                    "dish_name": dish_name,
                    "section": "Summary",
                }
            }
            chunks.append(summary_chunk)

            # Recursively add sections and subsections as chunks
            def add_sections_to_chunks(sections, parent_section=None, level=0):
                for s in sections:
                    # Create a chunk for the section
                    section_title = f"{parent_section} - {s.title}" if parent_section else s.title
                    section_chunk = {
                        "text": f"{s.text[:200]} Source: {page.canonicalurl}",  # Append URL
                        "metadata": {
                            "dish_name": dish_name,
                            "section": section_title,
                        }
                    }
                    chunks.append(section_chunk)

                    # Add subsections recursively, passing the current section as the parent
                    add_sections_to_chunks(s.sections, parent_section=s.title, level=level + 1)

            # Start processing sections
            add_sections_to_chunks(page.sections)

            return chunks

        # Create chunks from the page
        chunks = create_chunks_from_page(page_py, info)
        print("Chunks created!")

        # Generate embeddings for metadata only
        chunk_embeddings = []
        for i, chunk in enumerate(chunks):
            # Convert metadata to a string (e.g., JSON or concatenated string)
            metadata_str = f"{chunk['metadata']['dish_name']} {chunk['metadata']['section']}"
            
            # Generate embedding for the metadata string
            metadata_embedding = get_embedding(metadata_str)
            
            chunk_embeddings.append({
                "id": f"chunk_{info}{i+1}",  # Unique ID for each chunk
                "vector": metadata_embedding,  # Embedding of the metadata
                "metadata": chunk["metadata"],  # Original metadata
                "text": chunk["text"]  # Optional: Store text if needed for retrieval
            })
        print("Embeddings created!")
    
    # Initialize Chroma client
    chroma_client = chromadb.Client()

    # # Directory where the Chroma DB will be persisted
    # persist_directory = "./chroma_db"  # Replace with your desired directory

    # # Initialize the Chroma client with persistence 
    # chroma_client = chromadb.Client(
    #     Settings(
    #     persist_directory=persist_directory,  # Enable persistence
    #     chroma_db_impl="duckdb+parquet",      # Use DuckDB with Parquet for storage
    #     )
    # )

    # Create a collection (similar to a table in a traditional DB)
    collection_name = 'ExternalInformationTest2'
    try:
        collection = chroma_client.get_collection(name=collection_name)
    except:
        collection = chroma_client.create_collection(name=collection_name)

    if count == '[(0,)]':
        # Add embeddings to the collection
        for chunk in chunk_embeddings:
            collection.add(
                ids=[chunk["id"]],  # Unique ID for each chunk
                embeddings=[chunk["vector"]],  # Embedding of the metadata
                metadatas=[chunk["metadata"]],  # Original metadata
                documents=[chunk["text"]]  # Optional: Store text if needed for retrieval
            )

        print("Metadata embeddings stored in Chroma!")
        execute_query.invoke(insert_query)


    # Generate embedding for the user query
    query_embedding = get_embedding(' '.join(history)+' '+user_query)

    # Perform similarity search
    results = collection.query(
        query_embeddings=[query_embedding],  # Embedding of the user query
        n_results=5 # Number of results to return
    )

    relevant_documents = results['documents']

    return relevant_documents

In [17]:
result = extractData_loadData_performSemanticSearch('sushi', 'where did sushi originate?')

Chunks created!
Embeddings created!
Metadata embeddings stored in Chroma!


In [18]:
result

[[' Source: https://en.wikipedia.org/wiki/Sushi',
  'The earliest written mention of sushi in English described in the Oxford English Dictionary is in an 1893 book, A Japanese Interior, where it mentions sushi as "a roll of cold rice with fish, sea-weed Source: https://en.wikipedia.org/wiki/Sushi',
  'Until the early 19th century, sushi slowly changed with Japanese cuisine. The Japanese started eating three meals a day, rice was boiled instead of steamed, and of large importance was the development Source: https://en.wikipedia.org/wiki/Sushi',
  'During the Edo period (1603–1867), a third type of sushi, haya-zushi (早寿司、早ずし, "fast sushi"), was developed. Haya-zushi differed from earlier sushi in that instead of lactic fermentation of rice, vine Source: https://en.wikipedia.org/wiki/Sushi',
  ' Source: https://en.wikipedia.org/wiki/Sushi']]

### Method to classify user query as Tier 1, 2 or 3

In [19]:
import os
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain, PromptTemplate


def classify_query(user_query, history=[]):
    # Step 1: Define the prompt template
    prompt_template = PromptTemplate(
        input_variables=["history", "user_query"],
        template='''I have a scenario where I want to classify query into a category.

        Tier 1: Questions related to restaurants, finding food, or food trends.
        Examples:
        Which restaurants in San Francisco offer dishes with Impossible Meat? (Label - Tier 1)
        Give me a summary of the latest trends around desserts. (Tier 1)
        Which restaurants are known for sushi? (Tier 1)
        Compare the average menu price of vegan restaurants in LA vs. Mexican restaurants. (Tier 1)
        Which food can I find with peas?

        Tier 2: Questions about a dish or ingredients.
        Examples:
        Tell me about biryani. (Tier 2)
        What is the history of sushi? (Tier 2)
        Tell me the contents of sushi. (Tier 2)
        
        Tier 3: Questions that combine both restaurant-related and dish-related queries.
        Example:
        What is the history of sushi, and which restaurants in my area are known for it?

        Tell me which class the query falls into with just one word that is the class. Note – might include chat history which should not be considered for classification but only for context

        history - {history}
        query - {user_query}'''
    )

    # Step 2: Initialize the LLM
    llm = ChatOpenAI(model="gpt-4", temperature=0)  # Use "gpt-4" instead of "gpt-4o"

    # Step 3: Create the LLMChain
    chain = LLMChain(llm=llm, prompt=prompt_template)

    # Step 4: Pass the user query and history to the chain
    response = chain.run({"history": history, "user_query": user_query})

    return response

In [20]:
classify_query("What is italian cusine and where can i find it near me")

/var/folders/7_/9w2s326d5cx3n3gsd9wtv04h0000gn/T/ipykernel_22453/1204639571.py:37: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model="gpt-4", temperature=0)  # Use "gpt-4" instead of "gpt-4o"
/var/folders/7_/9w2s326d5cx3n3gsd9wtv04h0000gn/T/ipykernel_22453/1204639571.py:40: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)
/var/folders/7_/9w2s326d5cx3n3gsd9wtv04h0000gn/T/ipykernel_22453/1204639571.py:43: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instea

'Tier 3'

### Tier 1 - Generate sql query --> Query DB --> Return result in naturla language

In [21]:
examples = [
    {
        "input": "Which restaurants in San Francisco offer dishes with Impossible Meat?",
        "query": "SELECT restaurant_name, city FROM restaurants WHERE city = 'San Francisco' AND id IN (SELECT restaurant_id FROM menu_items WHERE menu_description ILIKE '%impossible%');"
    },
    {
        "input": "Find restaurants near me that serve gluten-free pizza.",
        "query": "SELECT restaurant_name FROM restaurants WHERE id IN (SELECT restaurant_id FROM menu_items WHERE menu_description ILIKE '%gluten free%' AND menu_item ILIKE '%pizza%');"
    },
    {
        "input": "Give me a summary of the latest trends around desserts.",
        "query": "SELECT mi.menu_item AS dessert_name, COUNT(r.id) AS restaurant_count, AVG(r.rating) AS avg_rating, SUM(r.review_count) AS total_reviews FROM menu_items mi JOIN restaurants r ON mi.restaurant_id = r.id WHERE mi.menu_category ILIKE '%dessert%' OR mi.categories ILIKE '%dessert%' GROUP BY mi.menu_item ORDER BY total_reviews DESC, avg_rating DESC LIMIT 10;"
    },
    {
        "input": "Which restaurants are known for Sushi?",
        "query": "SELECT r.restaurant_name, r.rating FROM restaurants r JOIN menu_items m ON r.id = m.restaurant_id WHERE m.menu_item ILIKE '%sushi%' AND r.rating >= 4.0 GROUP BY r.restaurant_name, r.rating ORDER BY r.rating DESC;"
    },
    {
        "input": "Compare the average menu price of vegan restaurants in LA vs. Mexican restaurants.",
        "query": "SELECT category, AVG(CASE WHEN price = '$' THEN 1 WHEN price = '$$' THEN 2 WHEN price = '$$$' THEN 3 WHEN price = '$$$$' THEN 4 END) AS avg_price_level FROM (SELECT r.id AS restaurant_id, CASE WHEN m.categories ILIKE '%vegan%' THEN 'Vegan' WHEN m.categories ILIKE '%mexican%' THEN 'Mexican' END AS category, r.price FROM restaurants r JOIN menu_items m ON r.id = m.restaurant_id WHERE r.city = 'Los Angeles' AND (m.categories ILIKE '%vegan%' OR m.categories ILIKE '%mexican%')) subquery GROUP BY category;"
    },
    {
        "input": "Top 5 famous desserts in Boston based on reviews.",
        "query": "SELECT m.menu_item, SUM(r.review_count) AS total_reviews FROM restaurants r JOIN menu_items m ON r.id = m.restaurant_id WHERE r.city = 'Boston' AND (m.menu_category ILIKE '%dessert%' OR m.categories ILIKE '%dessert%') GROUP BY m.menu_item ORDER BY total_reviews DESC LIMIT 5;"
    },
    {
        "input": "Average rating of Mexican and Italian restaurants.",
        "query": "SELECT CASE WHEN m.categories ILIKE '%mexican%' THEN 'Mexican' WHEN m.categories ILIKE '%italian%' THEN 'Italian' END AS cuisine, AVG(r.rating) AS avg_rating FROM restaurants r JOIN menu_items m ON r.id = m.restaurant_id AND (m.categories ILIKE '%mexican%' OR m.categories ILIKE '%italian%') GROUP BY cuisine;"
    },
    {
        "input": "Restaurants with the most vegetarian options.",
        "query": "SELECT r.restaurant_name, r.city, COUNT(m.item_id) AS vegetarian_item_count FROM restaurants r JOIN menu_items m ON r.id = m.restaurant_id WHERE m.categories ILIKE '%vegetarian%' GROUP BY r.restaurant_name, r.city ORDER BY vegetarian_item_count DESC LIMIT 5;"
    }
]

from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
import streamlit as st

@st.cache_resource
def get_example_selector():
    example_selector = SemanticSimilarityExampleSelector.from_examples(
        examples,
        OpenAIEmbeddings(),
        Chroma,
        k=3,
        input_keys=["input"],
    )
    return example_selector

In [22]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,FewShotChatMessagePromptTemplate,PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

def tierOne(user_query, history=[]):
    
    select_table = ['menu_items', 'restaurants']
    db = SQLDatabase.from_uri(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}", include_tables = select_table)
    llm = ChatOpenAI(model="gpt-4o", temperature=0)

    example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nSQLQuery:"),
        ("ai", "{query}"),
    ]
    )

    few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=get_example_selector(),
    input_variables=["input", 'top_k'],
    )

    final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a PostgreSQL expert. Given an input question, create a syntactically correct PostgreSQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries."),
        few_shot_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
    )

    answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

    Chat History:{history}
    Question: {question}
    SQL Query: {query}
    SQL Result: {result}
    Answer: """
    )

    generate_query = create_sql_query_chain(llm, db,final_prompt) 
    execute_query = QuerySQLDataBaseTool(db=db)

    rephrase_answer = answer_prompt | llm | StrOutputParser()

    chain = (
    RunnablePassthrough.assign(query=generate_query).assign(
        result=itemgetter("query") | execute_query
    )
    | rephrase_answer )

    return chain.invoke({'question':user_query, 'history':history})



In [23]:
answer = tierOne('Where can i find veg food')

2025-02-04 15:06:21.522 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-04 15:06:21.559 
  command:

    streamlit run /opt/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-02-04 15:06:21.562 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-04 15:06:21.562 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-04 15:06:22.069 Thread 'Thread-56': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-04 15:06:22.070 Thread 'Thread-56': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-04 15:06:22.419 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-04 15:06:22.420 Thread 'MainThread': mis

In [24]:
print(answer)

You can find vegetarian or vegan food at the following restaurants in San Francisco: Vegan Mob, Udupi Palace, Pinche Sushi, Beloved Cafe, Healthyish Republic, Mission Curry House, Cha-Ya San Francisco, and Indochine Vegan.


### Tier 2 

#### Extract ingredinet or dish from user query

In [25]:
import os
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain, PromptTemplate


def extract_info(user_query, history=[]):
    # Step 1: Define the prompt template
    prompt_template = PromptTemplate(
        input_variables=["history", "user_query"],
        template='''You know that the query below is asking information about some dish or ingredient. Identify the ingredient or dish name and return only that value.
        Examples -
        Tell me about biryani. Biryani 
        What is the history of sushi? Sushi
        Tell me about Saffron. Saffron
        Below is the query for you to identify and includes chat history if any,
        Note - Add a underscore if multiword 
        
        history - {history}
        query - {user_query}'''
    )

    # Step 2: Initialize the LLM
    llm = ChatOpenAI(model="gpt-4", temperature=0)  # Use "gpt-4" instead of "gpt-4o"

    # Step 3: Create the LLMChain
    chain = LLMChain(llm=llm, prompt=prompt_template)

    # Step 4: Pass the user query and history to the chain
    response = chain.run({"history": history, "user_query": user_query})

    return response

In [26]:
extract_info('What is the most common ingredients used in pasta')

'Pasta'

#### Use extractData_loadData_performSemanticSearch method to retrieve relevant info

In [27]:
query = 'What is the most common ingredients used in sushi'

In [28]:
context = extractData_loadData_performSemanticSearch(extract_info(query), query)

In [29]:
context

[['All sushi has a base of specially prepared rice, complemented with other ingredients. Traditional Japanese sushi consists of rice flavored with vinegar sauce and various raw or cooked ingredients. Source: https://en.wikipedia.org/wiki/Sushi',
  'Sushi-meshi (鮨飯) (also known as su-meshi (酢飯), shari (舎利), or gohan (ご飯)) is a preparation of white, short-grained, Japanese rice mixed with a dressing consisting of rice vinegar, sugar, salt, and occ Source: https://en.wikipedia.org/wiki/Sushi',
  'Sushi is commonly eaten with condiments. Sushi may be dipped in shōyu (soy sauce), and is usually flavored with wasabi, a piquant paste made from the grated stem of the Wasabia japonica plant. Japanes Source: https://en.wikipedia.org/wiki/Sushi',
  'The dark green seaweed wrappers used in makimono are called nori (海苔). Nori is a type of red algae, typically in the family Bangiaceae, traditionally cultivated in the harbors of Japan. Originally, al Source: https://en.wikipedia.org/wiki/Sushi',
  'T

#### Answer query using context found above and return result

In [30]:
def result_nl(context, user_query, history = []):
    
    prompt_template = PromptTemplate(
        input_variables=["context", "history", "user_query"],
        template='''Using relevant context passed answer the user query. If history of chat availableuse it for relevant context. Note - After answering the query provide relevant source
        Relevant Context - {context}
        history - {history}
        query - {user_query}'''
    )

     # Step 2: Initialize the LLM
    llm = ChatOpenAI(model="gpt-4", temperature=0)  # Use "gpt-4" instead of "gpt-4o"

    # Step 3: Create the LLMChain
    chain = LLMChain(llm=llm, prompt=prompt_template)

    # Step 4: Pass the user query and history to the chain
    response = chain.run({"context": context, "history": history, "user_query": user_query})

    return response

In [31]:
result_nl(context, query)

'The most common ingredients used in sushi are specially prepared rice known as Sushi-meshi, which is a preparation of white, short-grained, Japanese rice mixed with a dressing consisting of rice vinegar, sugar, salt. The ingredients used inside sushi are called gu and are typically varieties of fish. The minimum quality and freshness of fish to be eaten raw must be superior to that of fish which is to be cooked. Sushi is commonly eaten with condiments like soy sauce and wasabi, a piquant paste made from the grated stem of the Wasabia japonica plant. The dark green seaweed wrappers used in makimono are called nori, which is a type of red algae.\n\nSource: https://en.wikipedia.org/wiki/Sushi'

### Tier 3

#### Split Tier 3 Query into Tier 2 and 1 

In [32]:
def split_query(user_query, history=[]):
    prompt_template = PromptTemplate(
        input_variables=["history", "user_query"],
        template='''Tier 1: Questions related to restaurants, finding food, or food trends. 
        Examples: 
        Which restaurants in San Francisco offer dishes with Impossible Meat? (Label - Tier 1) 
        Give me a summary of the latest trends around desserts. (Tier 1) 
        Which restaurants are known for sushi? (Tier 1) 
        Compare the average menu price of vegan restaurants in LA vs. Mexican restaurants. (Tier 1) 

        Tier 2: Questions about a dish or ingredients. 
        Examples: 
        Tell me about biryani. (Tier 2) 	
        What is the history of sushi? (Tier 2) 
        Tell me the contents of sushi. (Tier 2)

        The query is a combination of both these classes.

        Break down the query I am passing to you below into Tier 1 and Tier 2 questions (with proper context included in both) in the same order, and separate them with commas.

        Example: 
        What is the history of sushi, and which restaurants are known for it?  
        Which restaurants are known for sushi?, What is the history of sushi?

        Dont include class names in split.

        history - {history}
        query - {user_query}'''
    )

     # Step 2: Initialize the LLM
    llm = ChatOpenAI(model="gpt-4", temperature=0)  # Use "gpt-4" instead of "gpt-4o"

    # Step 3: Create the LLMChain
    chain = LLMChain(llm=llm, prompt=prompt_template)

    # Step 4: Pass the user query and history to the chain
    response = chain.run({"history": history, "user_query": user_query})

    return response.split(',')

In [33]:
split_queries = split_query('What is biryani and where can i find it')

In [34]:
for query in split_queries:
    print(query)
    print(classify_query(query))

What is biryani?
Tier 2
 Where can I find biryani?
Tier 1


### Edge Cases

#### If query not relevant to food or ingredients or restaurant context

In [35]:
def check_query(user_query, history=[]):
    prompt_template = PromptTemplate(
        input_variables=["history", "user_query"],
        template='''My RAG application or chatbot is for answering questions related to restaurants, food, dishes. So basically a one stop solution to find your favourite food or 
        to know more about it. 

        So you are the gatekeeper checking if the query given by the user is in context of what we are trying to solve or anything else.

        If query is in context just return True if out of context return a funny response stating we might be able to answer that in future. Make sure you refer the history for previous 
        queries and responses to decide on the context. If you observe the user asking the same out of context question let him know about it in a funny way.

        Note -  Only return True that is one word for in context queries  
        Make sure you return Only funny message as a response if out of context no need to explain anything else

        history - {history}
        query - {user_query}'''
    )

     # Step 2: Initialize the LLM
    llm = ChatOpenAI(model="gpt-4", temperature=0)  # Use "gpt-4" instead of "gpt-4o"

    # Step 3: Create the LLMChain
    chain = LLMChain(llm=llm, prompt=prompt_template)

    # Step 4: Pass the user query and history to the chain
    response = chain.run({"history": history, "user_query": user_query})

    return response

In [42]:
if check_query('Best Sushi spot in town') == str(True): print('yes')

yes


In [44]:
response = "response_one" + '\n\n' + "response_two"
print(response)

response_one

response_two
